In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [2]:
from wikidocsloader import WikidocsLoader

In [3]:
book_id = 20
loader = WikidocsLoader(book_id)
documents = loader.load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 987, which is longer than the specified 600
Created a chunk of size 616, which is longer than the specified 600
Created a chunk of size 1487, which is longer than the specified 600
Created a chunk of size 659, which is longer than the specified 600


In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [6]:
search_index = FAISS.from_documents(docs, embeddings)

In [7]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0))

In [8]:
def ask(question):
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=3),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    )

In [9]:
ask('위키독스의 특징은 무엇인가요?')

 WikiDocs has many features, including simplicity, a table of contents, the ability to exchange feedback and comments with authors, unique URLs for each page, email notifications, collaborative editing, and the ability to back up the entire book.
SOURCES: https://wikidocs.net/151


In [10]:
ask('위키독스 API가 있나요?')

 Yes, Wikidocs provides an Open API. 
SOURCES: https://wikidocs.net/178030


In [11]:
ask('위키독스에 광고를 게시할 수 있나요?')

 Yes, Wikidocs can publish ads. The author's ads and Wikidocs' ads are displayed in a 9:1 ratio.
SOURCES: https://wikidocs.net/164693, https://wikidocs.net/153, https://wikidocs.net/151142, https://wikidocs.net/689
